In [1]:
pip install lxml


     |████████████████████████████████| 5.8MB 6.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install BeautifulSoup4

     |████████████████████████████████| 112kB 16.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from bs4 import BeautifulSoup
import requests

with open('List of postal codes of Canada.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [10]:
Postal_code=[]
Borough=[]
Neighbourhood=[]
for row in soup.find('tbody').find_all('tr'):
   cells = row.find_all('td')
   Postal_code.append(cells[0].text.rstrip('\n'))
   Borough.append(cells[1].text.rstrip('\n'))
   try:
        Neighbourhood.append(cells[2].text.rstrip('\n'))
   except:
        Neighbourhood.append('Not Assigned')

#### Constructing the Dataframe

In [23]:
import pandas as pd
df = pd.DataFrame({'PostalCode': Postal_code,
                  'Borough': Borough,
                  'Neighborhood': Neighbourhood})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Removing rows having "Not assigned" Boroughs

In [24]:
df=df[df.Borough!='Not assigned'].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


#### Merging rows having same Postalcode

In [25]:
df1=df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Assigning Borough to Neighborhood whose Neighborhood is not assigned

In [26]:
for index, row in df1.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood']=row['Borough']
df1

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [22]:
df1.shape

(103, 3)

In [27]:
df1.to_csv("xyz.csv")